In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import models, datasets, transforms
from torchinfo import summary

from module.train import fit, test_multi_classification
from module.utils import plot_fit_result

import os

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

# 하이퍼파라미터

In [ ]:
epochs = 5
batch_size = 40
lr = 0.001

save_best_model = True
save_model_path = r"C:\Documents\deeplearning\saved_models"
early_stopping = False
patience = 10
data_path = r"C:\Classes\deeplearning\datasets"

# Transformer, Dataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),  # 224 x 224
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_set = datasets.CIFAR10(data_path, train=True, transform=transform, download=True)
test_set = datasets.CIFAR10(data_path, train=False, transform=transform, download=True)

In [ ]:
print(train_set.classes)

# DataLoader

In [ ]:
train_loader = DataLoader(train_set, batch_size, shuffle=True, drop_last=True, num_workers=os.cpu_count())
test_loader = DataLoader(test_set, batch_size, num_workers=os.cpu_count())

### Image 확인

In [ ]:
import torchvision
import matplotlib.pyplot as plt
import numpy as np
images, labels = next(iter(train_loader))

# print(images.shape)
concat_img = torchvision.utils.make_grid(images)  # image들을 합쳐서 하나의 tensor로 만든다.
# print(concat_img.shape)

concat_img = concat_img / 2 + 0.5     # unnormalize,   normalize = x * 표준편차 + 평균
img_arr = concat_img.numpy()  # Tensor -> ndarray

plt.figure(figsize=(13, 6))
plt.imshow(np.transpose(img_arr, (1, 2, 0)))
plt.show()

# Modeld 정의

In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT) 
summary(model, (1, 3, 224, 224))

##  frozon

In [ ]:
# for param in model.parameters():
#     param.requires_grad = False

## classifier 변경

In [ ]:
print(model)

In [ ]:
in_features = model.fc.in_features
# print(in_features)

### 출력 Layer 변경
model.fc = nn.Linear(in_features, 10)

In [ ]:
summary(model, (1, 3, 224, 224))

#  학습

In [ ]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)  # momentum
scheduler = None

In [ ]:
result = fit(train_loader, test_loader, model, loss_fn, optimizer, 5, 
                save_best_model=save_best_model, save_model_path=os.path.join(save_model_path, "cifar10_resnet18.pth"),
                early_stopping=early_stopping,patience=patience, 
                mode="multi",
                device=device, lr_scheduler=scheduler)